In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [9]:
# This is required when using GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Fixed, no need change
def load_data(file):
    with open(file, 'rb') as fo:
        try:
            samples = pickle.load(fo)
        except UnicodeDecodeError:  # python 3.x
            fo.seek(0)
            samples = pickle.load(fo, encoding='latin1')

    data, labels = samples['data'], samples['labels']

    data = np.array(data, dtype=np.float32) / 255
    labels = np.array(labels, dtype=np.int32)
    return data, labels


def make_model(num_ch_c1, num_ch_c2, use_dropout):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(3072, )))
    model.add(layers.Reshape(target_shape=(32, 32, 3), input_shape=(3072,)))
    model.add(layers.Conv2D(filters=num_ch_c1, kernel_size=9, activation='relu', padding='valid', input_shape=(None, None, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
    model.add(layers.Conv2D(filters=num_ch_c2, kernel_size=5, activation='relu', padding='valid', input_shape=(None, None, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(300, activation=None, use_bias=True)) # ??
    model.add(layers.Dense(10, use_bias=True, input_shape=(300,)))  # Here no softmax because we have combined it with the loss
    return model

def save_model(model, use_dropout, num_ch_c1, num_ch_c2, optimizer_):
    # Create folder to store models and results
    if not os.path.exists('./models'):
        os.mkdir('./models')
    if not os.path.exists('./results'):
        os.mkdir('./results')

    # Save model
    if use_dropout:
        model.save(f'./models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout')
    else:
        model.save(f'./models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout')

def save_plot(history, use_dropout, num_ch_c1, num_ch_c2, optimizer_):
    # Save the plot for losses
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    test_acc = history.history['val_accuracy']
    plt.plot(range(1, len(train_loss) + 1), train_loss, label='Train')
    plt.plot(range(1, len(val_loss) + 1), val_loss, label='Test')
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()
    if use_dropout:
        plt.savefig(
            f'./results/{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_loss.pdf')
    else:
        plt.savefig(
            f'./results/{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_loss.pdf'
        )
    plt.close()

    # Save the plot for accuracies
    plt.plot(range(1, len(train_acc) + 1), train_acc, label='Train')
    plt.plot(range(1, len(test_acc) + 1), test_acc, label='Test')
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend()
    if use_dropout:
        plt.savefig(
            f'./results/{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_accuracy.pdf'
        )
    else:
        plt.savefig(
            f'./results/{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_accuracy.pdf'
        )
    plt.close()


In [5]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
models = {}

num_ch_c1 = 50  
num_ch_c2 = 60 

epochs = 10  # Fixed
batch_size = 128  # Fixed
learning_rate = 0.001
use_dropout = False

model = make_model(num_ch_c1, num_ch_c2, use_dropout)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

# Training and test
x_train, y_train = load_data('./data/data_batch_1')
x_test, y_test = load_data('./data/test_batch_trim')

# Training
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test))


Epoch 1/10
79/79 [==============================] - 6s 78ms/step - loss: 2.2918 - accuracy: 0.1390 - val_loss: 2.2789 - val_accuracy: 0.1555
Epoch 2/10
79/79 [==============================] - 6s 77ms/step - loss: 2.2683 - accuracy: 0.1588 - val_loss: 2.2613 - val_accuracy: 0.1800
Epoch 3/10
79/79 [==============================] - 6s 81ms/step - loss: 2.2505 - accuracy: 0.1723 - val_loss: 2.2441 - val_accuracy: 0.1970
Epoch 4/10
79/79 [==============================] - 7s 83ms/step - loss: 2.2319 - accuracy: 0.1892 - val_loss: 2.2254 - val_accuracy: 0.1990
Epoch 5/10
79/79 [==============================] - 6s 80ms/step - loss: 2.2117 - accuracy: 0.2021 - val_loss: 2.2040 - val_accuracy: 0.2105
Epoch 6/10
79/79 [==============================] - 6s 80ms/step - loss: 2.1899 - accuracy: 0.2107 - val_loss: 2.1822 - val_accuracy: 0.2310
Epoch 7/10
79/79 [==============================] - 6s 79ms/step - loss: 2.1675 - accuracy: 0.2236 - val_loss: 2.1600 - val_accuracy: 0.2365
Epoch 8/10
79

In [11]:
# save model and the plots
save_model(model, use_dropout, num_ch_c1, num_ch_c2, 'sgd')
save_plot(history, use_dropout, num_ch_c1, num_ch_c2, 'sgd')

INFO:tensorflow:Assets written to: ./models/50_60_sgd_no_dropout/assets
